In [703]:
from matplotlib import pyplot as plt
import plotly.graph_objects as go
from datetime import datetime
import yfinance as yf
import pandas as pd

plt.style.use('Solarize_Light2')

In [704]:
df = pd.read_csv('data/WALCL.csv')

df['DATE'] = pd.to_datetime(df['DATE'])
df['WALCL'] = df['WALCL']
df.set_index('DATE', inplace=True)

### Plotting data

In [705]:
fig = go.Figure(go.Scatter(x = df.index, y = df['WALCL']/10e5, fill='tozeroy', name='WALCL'))

fig.update_layout(
    title=dict(text='Central Bank (Fed) Balance Sheet'),
    plot_bgcolor='rgb(230, 230,230)',
    yaxis_title='Trillions of dollars',
    width=1400, height=500
)

fig.show()

### Central bank balance sheet vs s&p500

- Load S&P500 data

In [706]:
today = datetime.today()

sp500_weekly = yf.download("^GSPC", start="2002-12-18", end=f"{today.year}-{today.month}-{today.day}", interval="1wk")

sp500_weekly.drop(columns=['Open', 'High', 'Low', 'Adj Close', 'Volume'], inplace=True)

[*********************100%***********************]  1 of 1 completed


In [707]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = df.index, y = (df['WALCL'])/1400, fill='tozeroy', name='WALCL'))

fig.add_trace(go.Scatter(x = sp500_weekly.index, y = sp500_weekly['Close'], name='S&P500 Close'))

fig.update_layout(
    title='S&P500 index vs Central Bank (Fed) Balance Sheet',
    plot_bgcolor='rgb(230, 230,230)',
    xaxis_title='',
    yaxis_title='$',
    showlegend=True,
    width=1400, height=600
)

fig.show()

In [708]:
# fig, ax1 = plt.subplots(figsize=(30, 10))

# ax1.plot(sp500_weekly.index, sp500_weekly['Close'] * 100, 'g-', label='S&P500')
# ax1.set_xlabel('')
# ax1.set_ylabel('S&P500 index', color='g')
# ax1.tick_params(axis='y', labelcolor='g')

# ax2 = ax1.twinx()

# ax2.plot(df.index, df['WALCL'].ffill(), 'b-', label='WALCL')
# ax2.fill_between(df.index, df['WALCL'].ffill(), alpha=.3, label='WALCL')
# ax2.set_ylabel('Central bank balance sheet', color='b')
# ax2.tick_params(axis='y', labelcolor='b')

# # Show the plot
# plt.title('S&P500 index vs Central bank balance sheet')
# plt.show();

### Central bank balance sheet to GDP ratio

In [709]:
GDP_df = pd.read_csv('data/GDP.csv')

GDP_df['DATE'] = pd.to_datetime(GDP_df['DATE'])
GDP_df.set_index('DATE', inplace=True)

In [710]:
monthly_df = df.resample('ME').min()
monthly_df.index = monthly_df.index.to_period('M').to_timestamp()

In [711]:
GDP_WALCL_ratio = pd.merge(GDP_df, monthly_df, left_index=True, right_index=True, how='inner')

GDP_WALCL_ratio['WALCL/GDP'] = 100 * GDP_WALCL_ratio['WALCL'] / (GDP_WALCL_ratio['GDP'] * 1000)

In [712]:
fig = go.Figure(go.Scatter(x = GDP_WALCL_ratio.index, y = GDP_WALCL_ratio['WALCL/GDP'], fill='tozeroy', name='WALCL to GDP ratio'))

fig.update_layout(
    title=dict(text='Central Bank (Fed) Balance Sheet as % of GDP'),
    plot_bgcolor='rgb(230, 230,230)',
    yaxis_title='%',
    width=1400, height=600
)

fig.show()